# Anna KaRNNa

In this notebook, I'll build a character-wise RNN trained on Anna Karenina, one of my all-time favorite books. It'll be able to generate new text based on the text from the book.

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Also, some information [here at r2rt](http://r2rt.com/recurrent-neural-networks-in-tensorflow-ii.html) and from [Sherjil Ozair](https://github.com/sherjilozair/char-rnn-tensorflow) on GitHub. Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

In [1]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

First we'll load the text file and convert it into integers for our network to use. Here I'm creating a couple dictionaries to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [2]:
# Load the entire text. Each character is turned into an integer
with open('anna.txt', 'r') as f:
    text=f.read()
vocab = sorted(set(text))
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

Let's check out the first 100 characters, make sure everything is peachy. According to the [American Book Review](http://americanbookreview.org/100bestlines.asp), this is the 6th best first line of a book ever.

In [3]:
text[:100]

'Chapter 1\r\n\r\n\r\nHappy families are all alike; every unhappy family is unhappy in its own\r\nway.\r\n\r\nEve'

And we can see the characters encoded as integers.

In [4]:
encoded[:100]

array([32, 65, 58, 73, 77, 62, 75,  2, 17,  1,  0,  1,  0,  1,  0, 37, 58,
       73, 73, 82,  2, 63, 58, 70, 66, 69, 66, 62, 76,  2, 58, 75, 62,  2,
       58, 69, 69,  2, 58, 69, 66, 68, 62, 27,  2, 62, 79, 62, 75, 82,  2,
       78, 71, 65, 58, 73, 73, 82,  2, 63, 58, 70, 66, 69, 82,  2, 66, 76,
        2, 78, 71, 65, 58, 73, 73, 82,  2, 66, 71,  2, 66, 77, 76,  2, 72,
       80, 71,  1,  0, 80, 58, 82, 14,  1,  0,  1,  0, 34, 79, 62], dtype=int32)

Since the network is working with individual characters, it's similar to a classification problem in which we are trying to predict the next character from the previous text.  Here's how many 'classes' our network has to pick from.

In [5]:
len(vocab)

84

## Making training mini-batches

Here is where we'll make our mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps. Considering a simple example, our batches would look like this:

<img src="assets/sequence_batching@1x.png" width=500px>


<br>
We have our text encoded as integers as one long array in `encoded`. Let's create a function that will give us an iterator for our batches. I like using [generator functions](https://jeffknupp.com/blog/2013/04/07/improve-your-python-yield-and-generators-explained/) to do this. Then we can pass `encoded` into this function and get our batch generator.

The first thing we need to do is discard some of the text so we only have completely full batches. Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences) and $M$ is the number of steps. Then, to get the number of batches we can make from some array `arr`, you divide the length of `arr` by the batch size. Once you know the number of batches and the batch size, you can get the total number of characters to keep.

After that, we need to split `arr` into $N$ sequences. You can do this using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences (`n_seqs` below), let's make that the size of the first dimension. For the second dimension, you can use `-1` as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$ where $K$ is the number of batches.

Now that we have this array, we can iterate through it to get our batches. The idea is each batch is a $N \times M$ window on the array. For each subsequent batch, the window moves over by `n_steps`. We also want to create both the input and target arrays. Remember that the targets are the inputs shifted over one character. You'll usually see the first input character used as the last target character, so something like this:
```python
y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
```
where `x` is the input batch and `y` is the target batch.

The way I like to do this window is use `range` to take steps of size `n_steps` from $0$ to `arr.shape[1]`, the total number of steps in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `n_steps` wide.

In [6]:
# We want to make a function that gives us a list of length n_seqs (batch-size).
# Each list is of length n_steps and corresponds to a consecutive set of characters

def get_batches(arr, n_seqs, n_steps):
    '''Create a generator that returns batches of size
       n_seqs x n_steps from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       n_seqs: Batch size, the number of sequences per batch
       n_steps: Number of sequence steps per batch
    '''
    # Get the number of characters per batch and number of batches we can make
    characters_per_batch = n_seqs * n_steps
    n_batches = len(arr)//characters_per_batch
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * characters_per_batch]
    
    # Reshape into n_seqs rows
    arr = arr.reshape((n_seqs, -1))
    
    for n in range(0, arr.shape[1], n_steps):
        # The features
        x = arr[:, n:n+n_steps]
        # The targets, shifted by one
        y = np.zeros_like(x)
        y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
        yield x, y

Now I'll make my data sets and we can check out what's going on here. Here I'm going to use a batch size of 10 and 50 sequence steps.

In [7]:
batches = get_batches(encoded, 10, 50)
x, y = next(batches)

In [8]:
print('x\n', x)
print('\ny\n', y)

('x\n', array([[32, 65, 58, 73, 77, 62, 75,  2, 17,  1,  0,  1,  0,  1,  0, 37, 58,
        73, 73, 82,  2, 63, 58, 70, 66, 69, 66, 62, 76,  2, 58, 75, 62,  2,
        58, 69, 69,  2, 58, 69, 66, 68, 62, 27,  2, 62, 79, 62, 75, 82],
       [ 2, 76, 69, 62, 62, 79, 62,  2, 72, 63,  2, 65, 66, 76,  2, 61, 75,
        62, 76, 76,  2, 60, 72, 58, 77, 12,  2,  4, 38,  8, 79, 62,  2, 76,
        78, 60, 65,  2, 58, 71,  2, 66, 61, 62, 58,  2, 63, 72, 75,  2],
       [62, 75, 14,  4,  1,  0,  1,  0, 48, 77, 62, 73, 58, 71,  2, 30, 75,
        68, 58, 61, 82, 62, 79, 66, 77, 60, 65,  2, 76, 70, 66, 69, 62, 61,
         2, 65, 58, 75, 61, 69, 82,  2, 73, 62, 75, 60, 62, 73, 77, 66],
       [76,  2, 70, 62, 58, 71, 66, 71, 64,  1,  0, 60, 69, 62, 58, 75, 69,
        82, 12,  2, 72, 75,  2, 59, 62, 60, 58, 78, 76, 62,  2, 65, 66, 76,
         2, 59, 75, 72, 77, 65, 62, 75,  2, 80, 72, 78, 69, 61,  2, 71],
       [65, 62, 75, 12,  2, 35, 66, 71, 72, 64, 62, 71,  2, 80, 58, 71, 77,
        76,  2, 

In [9]:
# Visualize this as the actual characters

print(["".join([int_to_vocab[el] for el in arr]) for arr in x])
print("")

x, y = next(batches)
print(["".join([int_to_vocab[el] for el in arr]) for arr in x])

['Chapter 1\r\n\r\n\r\nHappy families are all alike; every', ' sleeve of his dress coat, "I\'ve such an idea for ', 'er."\r\n\r\nStepan Arkadyevitch smiled hardly percepti', 's meaning\r\nclearly, or because his brother would n', 'her, Finogen wants some tar," said the young woman', 'd\r\nso he had decided that so it must be. But he ha', 'icing her husband\'s face of dismay, "or go on;\r\ngo', 'en offended at his jealousy; she was\r\nangry that t', 's talking at the high table.\r\n\r\n"That\'s the new ge', 'ame circle that she had been round so often before']

[' unhappy family is unhappy in its own\r\nway.\r\n\r\nEve', 'a _cotillion! Un\r\nbijou!_"\r\n\r\nAnd he moved gradual', 'bly, catching the\r\ninstantaneous change he knew so', 'ot or could not understand him.\r\nBut he did not pu', ' in the clogs,\r\ncoming in.\r\n\r\n"Yes, yes, that\'s ho', 'd not realized what an\r\neffect it would have on he', ', and then come for me," she said, and went back i', 'he slightest amusement, even

If you implemented `get_batches` correctly, the above output should look something like 
```
x
 [[55 63 69 22  6 76 45  5 16 35]
 [ 5 69  1  5 12 52  6  5 56 52]
 [48 29 12 61 35 35  8 64 76 78]
 [12  5 24 39 45 29 12 56  5 63]
 [ 5 29  6  5 29 78 28  5 78 29]
 [ 5 13  6  5 36 69 78 35 52 12]
 [63 76 12  5 18 52  1 76  5 58]
 [34  5 73 39  6  5 12 52 36  5]
 [ 6  5 29 78 12 79  6 61  5 59]
 [ 5 78 69 29 24  5  6 52  5 63]]

y
 [[63 69 22  6 76 45  5 16 35 35]
 [69  1  5 12 52  6  5 56 52 29]
 [29 12 61 35 35  8 64 76 78 28]
 [ 5 24 39 45 29 12 56  5 63 29]
 [29  6  5 29 78 28  5 78 29 45]
 [13  6  5 36 69 78 35 52 12 43]
 [76 12  5 18 52  1 76  5 58 52]
 [ 5 73 39  6  5 12 52 36  5 78]
 [ 5 29 78 12 79  6 61  5 59 63]
 [78 69 29 24  5  6 52  5 63 76]]
 ```
 although the exact numbers will be different. Check to make sure the data is shifted over one step for `y`.

## Building the model

Below is where you'll build the network. We'll break it up into parts so it's easier to reason about each bit. Then we can connect them up into the whole network.

<img src="assets/charRNN.png" width=500px>


### Inputs

First off we'll create our input placeholders. As usual we need placeholders for the training data and the targets. We'll also create a placeholder for dropout layers called `keep_prob`.

In [10]:
def build_inputs(batch_size, num_steps):
    ''' Define placeholders for inputs, targets, and dropout 
    
        Arguments
        ---------
        batch_size: Batch size, number of sequences per batch
        num_steps: Number of sequence steps in a batch
        
    '''
    # Declare placeholders we'll feed into the graph
    inputs = tf.placeholder(tf.int32, [batch_size, num_steps], name='inputs')
    targets = tf.placeholder(tf.int32, [batch_size, num_steps], name='targets')
    
    # Keep probability placeholder for drop out layers
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    return inputs, targets, keep_prob

### LSTM Cell

Here we will create the LSTM cell we'll use in the hidden layer. We'll use this cell as a building block for the RNN. So we aren't actually defining the RNN here, just the type of cell we'll use in the hidden layer.

We first create a basic LSTM cell with

```python
lstm = tf.contrib.rnn.BasicLSTMCell(num_units)
```

where `num_units` is the number of units in the hidden layers in the cell. Then we can add dropout by wrapping it with 

```python
tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
```
You pass in a cell and it will automatically add dropout to the inputs or outputs. Finally, we can stack up the LSTM cells into layers with [`tf.contrib.rnn.MultiRNNCell`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/rnn/MultiRNNCell). With this, you pass in a list of cells and it will send the output of one cell into the next cell. Previously with TensorFlow 1.0, you could do this

```python
tf.contrib.rnn.MultiRNNCell([cell]*num_layers)
```

This might look a little weird if you know Python well because this will create a list of the same `cell` object. However, TensorFlow 1.0 will create different weight matrices for all `cell` objects. But, starting with TensorFlow 1.1 you actually need to create new cell objects in the list. To get it to work in TensorFlow 1.1, it should look like

```python
def build_cell(num_units, keep_prob):
    lstm = tf.contrib.rnn.BasicLSTMCell(num_units)
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    return drop
    
tf.contrib.rnn.MultiRNNCell([build_cell(num_units, keep_prob) for _ in range(num_layers)])
```

Even though this is actually multiple LSTM cells stacked on each other, you can treat the multiple layers as one cell.

We also need to create an initial cell state of all zeros. This can be done like so

```python
initial_state = cell.zero_state(batch_size, tf.float32)
```

Below, we implement the `build_lstm` function to create these LSTM cells and the initial state.

In [11]:
def build_lstm(lstm_size, num_layers, batch_size, keep_prob):
    ''' Build LSTM cell.
    
        Arguments
        ---------
        keep_prob: Scalar tensor (tf.placeholder) for the dropout keep probability
        lstm_size: Size of the hidden layers in the LSTM cells
        num_layers: Number of LSTM layers
        batch_size: Batch size

    '''
    ### Build the LSTM Cell
    
    def build_cell(lstm_size, keep_prob):
        # Use a basic LSTM cell
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
        
        # Add dropout to the cell
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return drop
    
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([build_cell(lstm_size, keep_prob) for _ in range(num_layers)])
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    return cell, initial_state

### RNN Output

Here we'll create the output layer. We need to connect the output of the RNN cells to a full connected layer with a softmax output. The softmax output gives us a probability distribution we can use to predict the next character.

If our input has batch size $N$, number of steps $M$, and the hidden layer has $L$ hidden units, then the output is a 3D tensor with size $N \times M \times L$: the output of each LSTM cell has size $L$, we have $M$ of them, one for each sequence step, and we have $N$ sequences. So the total size is $N \times M \times L$.

We are using the same fully connected layer, the same weights, for each of the outputs. Then, to make things easier, we should reshape the outputs into a 2D tensor with shape $(M * N) \times L$. That is, one row for each sequence and step, where the values of each row are the output from the LSTM cells.

One we have the outputs reshaped, we can do the matrix multiplication with the weights. We need to wrap the weight and bias variables in a variable scope with `tf.variable_scope(scope_name)` because there are weights being created in the LSTM cells. TensorFlow will throw an error if the weights created here have the same names as the weights created in the LSTM cells, which they will be default. To avoid this, we wrap the variables in a variable scope so we can give them unique names.

In [ ]:
def build_output(lstm_output, in_size, out_size):
    ''' Build a softmax layer, return the softmax output and logits.
    
        Arguments
        ---------
        
        x: Input tensor
        in_size: Size of the input tensor, for example, size of the LSTM cells (=width of LSTM cell layer)
        out_size: Size of this softmax layer
    
    '''

    # Reshape output so it's a bunch of rows, one row for each step for each sequence.
    # That is, the shape should be batch_size*num_steps rows by lstm_size columns
    seq_output = tf.concat(lstm_output, axis=1)
    x = tf.reshape(seq_output, [-1, in_size])
    
    # Connect the RNN outputs to a softmax layer
    with tf.variable_scope('softmax'):
        softmax_w = tf.Variable(tf.truncated_normal((in_size, out_size), stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(out_size))
    
    # Since output is a bunch of rows of RNN cell outputs, logits will be a bunch
    # of rows of logit outputs, one for each step and sequence
    logits = tf.matmul(x, softmax_w) + softmax_b
    
    # Use softmax to get the probabilities for predicted characters
    out = tf.nn.softmax(logits, name='predictions')
    
    return out, logits

### Training loss

Next up is the training loss. We get the logits and targets and calculate the softmax cross-entropy loss. First we need to one-hot encode the targets, we're getting them as encoded characters. Then, reshape the one-hot targets so it's a 2D tensor with size $(M*N) \times C$ where $C$ is the number of classes/characters we have. Remember that we reshaped the LSTM outputs and ran them through a fully connected layer with $C$ units. So our logits will also have size $(M*N) \times C$.

Then we run the logits and targets through `tf.nn.softmax_cross_entropy_with_logits` and find the mean to get the loss.

In [ ]:
def build_loss(logits, targets, lstm_size, num_classes):
    ''' Calculate the loss from the logits and the targets.
    
        Arguments
        ---------
        logits: Logits from final fully connected layer
        targets: Targets for supervised learning (can be ints, or characters)
        lstm_size: Number of LSTM hidden units
        num_classes: Number of classes in targets
        
    '''
    
    # One-hot encode targets and reshape to match logits, one row per batch_size per step
    y_one_hot = tf.one_hot(targets, num_classes)
    y_reshaped = tf.reshape(y_one_hot, logits.get_shape())
    
    # Softmax cross entropy loss
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped)
    loss = tf.reduce_mean(loss)
    return loss

### Optimizer

Here we build the optimizer. Normal RNNs have issues with gradients exploding and disappearing. LSTMs fix the disappearance problem, but the gradients can still grow without bound. To fix this, we can clip the gradients above some threshold. That is, if a gradient is larger than that threshold, we set it to the threshold. This will ensure the gradients never grow overly large. Then we use an AdamOptimizer for the learning step.

In [14]:
def build_optimizer(loss, learning_rate, grad_clip):
    ''' Build optmizer for training, using gradient clipping.
    
        Arguments:
        loss: Network loss
        learning_rate: Learning rate for optimizer
    
    '''
    
    # Optimizer for training, using gradient clipping to control exploding gradients
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

### Build the network

Now we can put all the pieces together and build a class for the network. To actually run data through the LSTM cells, we will use [`tf.nn.dynamic_rnn`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/nn/dynamic_rnn). This function will pass the hidden and cell states across LSTM cells appropriately for us. It returns the outputs for each LSTM cell at each step for each sequence in the mini-batch. It also gives us the final LSTM state. We want to save this state as `final_state` so we can pass it to the first LSTM cell in the the next mini-batch run. For `tf.nn.dynamic_rnn`, we pass in the cell and initial state we get from `build_lstm`, as well as our input sequences. Also, we need to one-hot encode the inputs before going into the RNN. 

In [15]:
class CharRNN:
    
    def __init__(self, num_classes, batch_size=64, num_steps=50, 
                       lstm_size=128, num_layers=2, learning_rate=0.001, 
                       grad_clip=5, sampling=False):
    
        # When we're using this network for sampling later, we'll be passing in
        # one character at a time, so providing an option for that
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps

        tf.reset_default_graph()
        
        # Build the input placeholder tensors
        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_steps)

        # Build the LSTM cell
        cell, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, self.keep_prob)

        ### Run the data through the RNN layers
        # First, one-hot encode the input tokens
        x_one_hot = tf.one_hot(self.inputs, num_classes)
        
        # Run each sequence step through the RNN and collect the outputs
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=self.initial_state)
        self.final_state = state
        
        # Get softmax predictions and logits
        self.prediction, self.logits = build_output(outputs, lstm_size, num_classes)
        
        # Loss and optimizer (with gradient clipping)
        self.loss = build_loss(self.logits, self.targets, lstm_size, num_classes)
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)

## Hyperparameters

Here I'm defining the hyperparameters for the network. 

* `batch_size` - Number of sequences running through the network in one pass.
* `num_steps` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lstm_size` - The number of units in the hidden layers.
* `num_layers` - Number of hidden LSTM layers to use
* `learning_rate` - Learning rate for training
* `keep_prob` - The dropout keep probability when training. If you're network is overfitting, try decreasing this.

Here's some good advice from Andrej Karpathy on training the network. I'm going to copy it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `lstm_size` and `num_layers`. I would advise that you always use `num_layers` of either 2/3. The `lstm_size` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `lstm_size` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.


In [16]:
batch_size = 100        # Sequences per batch
num_steps = 100         # Number of sequence steps per batch
lstm_size = 512         # Size of hidden layers in LSTMs
num_layers = 2          # Number of LSTM layers
learning_rate = 0.001   # Learning rate
keep_prob = 0.5         # Dropout keep probability

## Time for training

This is typical training code, passing inputs and targets into the network, then running the optimizer. Here we also get back the final LSTM state for the mini-batch. Then, we pass that state back into the network so the next batch can continue the state from the previous batch. And every so often (set by `save_every_n`) I save a checkpoint.

Here I'm saving checkpoints with the format

`i{iteration number}_l{# hidden layer units}.ckpt`

In [17]:
epochs = 20
# Save every N iterations
save_every_n = 200

model = CharRNN(len(vocab), batch_size=batch_size, num_steps=num_steps,
                lstm_size=lstm_size, num_layers=num_layers, 
                learning_rate=learning_rate)

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/______.ckpt')
    counter = 0
    for e in range(epochs):
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in get_batches(encoded, batch_size, num_steps):
            counter += 1
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            
            end = time.time()
            print('Epoch: {}/{}... '.format(e+1, epochs),
                  'Training Step: {}... '.format(counter),
                  'Training loss: {:.4f}... '.format(batch_loss),
                  '{:.4f} sec/batch'.format((end-start)))
        
            if (counter % save_every_n == 0):
                saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))
    
    saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

('Epoch: 1/20... ', 'Training Step: 1... ', 'Training loss: 4.4318... ', '6.8468 sec/batch')
('Epoch: 1/20... ', 'Training Step: 2... ', 'Training loss: 4.3368... ', '6.1452 sec/batch')
('Epoch: 1/20... ', 'Training Step: 3... ', 'Training loss: 3.8219... ', '6.0816 sec/batch')
('Epoch: 1/20... ', 'Training Step: 4... ', 'Training loss: 5.3987... ', '6.1062 sec/batch')
('Epoch: 1/20... ', 'Training Step: 5... ', 'Training loss: 4.3476... ', '6.5625 sec/batch')
('Epoch: 1/20... ', 'Training Step: 6... ', 'Training loss: 4.0597... ', '7.0476 sec/batch')
('Epoch: 1/20... ', 'Training Step: 7... ', 'Training loss: 3.8347... ', '6.5046 sec/batch')
('Epoch: 1/20... ', 'Training Step: 8... ', 'Training loss: 3.6578... ', '6.8995 sec/batch')
('Epoch: 1/20... ', 'Training Step: 9... ', 'Training loss: 3.5760... ', '7.1578 sec/batch')
('Epoch: 1/20... ', 'Training Step: 10... ', 'Training loss: 3.4765... ', '6.0830 sec/batch')
('Epoch: 1/20... ', 'Training Step: 11... ', 'Training loss: 3.4206..

('Epoch: 1/20... ', 'Training Step: 89... ', 'Training loss: 3.1546... ', '6.1255 sec/batch')
('Epoch: 1/20... ', 'Training Step: 90... ', 'Training loss: 3.1350... ', '6.2005 sec/batch')
('Epoch: 1/20... ', 'Training Step: 91... ', 'Training loss: 3.1250... ', '6.2151 sec/batch')
('Epoch: 1/20... ', 'Training Step: 92... ', 'Training loss: 3.1497... ', '6.1027 sec/batch')
('Epoch: 1/20... ', 'Training Step: 93... ', 'Training loss: 3.1310... ', '6.1392 sec/batch')
('Epoch: 1/20... ', 'Training Step: 94... ', 'Training loss: 3.1427... ', '5.9819 sec/batch')
('Epoch: 1/20... ', 'Training Step: 95... ', 'Training loss: 3.1201... ', '6.1003 sec/batch')
('Epoch: 1/20... ', 'Training Step: 96... ', 'Training loss: 3.1248... ', '6.3364 sec/batch')
('Epoch: 1/20... ', 'Training Step: 97... ', 'Training loss: 3.1200... ', '6.1883 sec/batch')
('Epoch: 1/20... ', 'Training Step: 98... ', 'Training loss: 3.0945... ', '6.1473 sec/batch')
('Epoch: 1/20... ', 'Training Step: 99... ', 'Training loss:

('Epoch: 1/20... ', 'Training Step: 176... ', 'Training loss: 2.6520... ', '6.5063 sec/batch')
('Epoch: 1/20... ', 'Training Step: 177... ', 'Training loss: 2.6339... ', '6.1598 sec/batch')
('Epoch: 1/20... ', 'Training Step: 178... ', 'Training loss: 2.6748... ', '6.1321 sec/batch')
('Epoch: 1/20... ', 'Training Step: 179... ', 'Training loss: 2.6523... ', '6.1223 sec/batch')
('Epoch: 1/20... ', 'Training Step: 180... ', 'Training loss: 2.6420... ', '6.4444 sec/batch')
('Epoch: 1/20... ', 'Training Step: 181... ', 'Training loss: 2.5866... ', '6.2354 sec/batch')
('Epoch: 1/20... ', 'Training Step: 182... ', 'Training loss: 2.6021... ', '6.3096 sec/batch')
('Epoch: 1/20... ', 'Training Step: 183... ', 'Training loss: 2.5545... ', '6.1532 sec/batch')
('Epoch: 1/20... ', 'Training Step: 184... ', 'Training loss: 2.5505... ', '5.9212 sec/batch')
('Epoch: 1/20... ', 'Training Step: 185... ', 'Training loss: 2.5546... ', '5.9852 sec/batch')
('Epoch: 1/20... ', 'Training Step: 186... ', 'Tra

('Epoch: 2/20... ', 'Training Step: 263... ', 'Training loss: 2.3291... ', '5.9687 sec/batch')
('Epoch: 2/20... ', 'Training Step: 264... ', 'Training loss: 2.2929... ', '6.1611 sec/batch')
('Epoch: 2/20... ', 'Training Step: 265... ', 'Training loss: 2.3076... ', '6.0965 sec/batch')
('Epoch: 2/20... ', 'Training Step: 266... ', 'Training loss: 2.2865... ', '6.1208 sec/batch')
('Epoch: 2/20... ', 'Training Step: 267... ', 'Training loss: 2.3138... ', '6.3183 sec/batch')
('Epoch: 2/20... ', 'Training Step: 268... ', 'Training loss: 2.3279... ', '6.3294 sec/batch')
('Epoch: 2/20... ', 'Training Step: 269... ', 'Training loss: 2.2865... ', '6.2603 sec/batch')
('Epoch: 2/20... ', 'Training Step: 270... ', 'Training loss: 2.2892... ', '6.1221 sec/batch')
('Epoch: 2/20... ', 'Training Step: 271... ', 'Training loss: 2.2595... ', '6.2893 sec/batch')
('Epoch: 2/20... ', 'Training Step: 272... ', 'Training loss: 2.3002... ', '6.2072 sec/batch')
('Epoch: 2/20... ', 'Training Step: 273... ', 'Tra

('Epoch: 2/20... ', 'Training Step: 350... ', 'Training loss: 2.1399... ', '6.2491 sec/batch')
('Epoch: 2/20... ', 'Training Step: 351... ', 'Training loss: 2.1522... ', '6.2843 sec/batch')
('Epoch: 2/20... ', 'Training Step: 352... ', 'Training loss: 2.1322... ', '6.2184 sec/batch')
('Epoch: 2/20... ', 'Training Step: 353... ', 'Training loss: 2.1623... ', '6.1322 sec/batch')
('Epoch: 2/20... ', 'Training Step: 354... ', 'Training loss: 2.1600... ', '5.9815 sec/batch')
('Epoch: 2/20... ', 'Training Step: 355... ', 'Training loss: 2.1672... ', '6.3599 sec/batch')
('Epoch: 2/20... ', 'Training Step: 356... ', 'Training loss: 2.1443... ', '6.3232 sec/batch')
('Epoch: 2/20... ', 'Training Step: 357... ', 'Training loss: 2.1398... ', '7.2658 sec/batch')
('Epoch: 2/20... ', 'Training Step: 358... ', 'Training loss: 2.1418... ', '6.3125 sec/batch')
('Epoch: 2/20... ', 'Training Step: 359... ', 'Training loss: 2.1383... ', '6.1782 sec/batch')
('Epoch: 2/20... ', 'Training Step: 360... ', 'Tra

...

('Epoch: 7/20... ', 'Training Step: 1303... ', 'Training loss: 1.4654... ', '5.9803 sec/batch')
('Epoch: 7/20... ', 'Training Step: 1304... ', 'Training loss: 1.4444... ', '5.9792 sec/batch')
('Epoch: 7/20... ', 'Training Step: 1305... ', 'Training loss: 1.5023... ', '6.3857 sec/batch')
('Epoch: 7/20... ', 'Training Step: 1306... ', 'Training loss: 1.4819... ', '6.2454 sec/batch')
('Epoch: 7/20... ', 'Training Step: 1307... ', 'Training loss: 1.4991... ', '6.0568 sec/batch')
('Epoch: 7/20... ', 'Training Step: 1308... ', 'Training loss: 1.4777... ', '5.8955 sec/batch')
('Epoch: 7/20... ', 'Training Step: 1309... ', 'Training loss: 1.4896... ', '5.9856 sec/batch')
('Epoch: 7/20... ', 'Training Step: 1310... ', 'Training loss: 1.4937... ', '6.0718 sec/batch')
('Epoch: 7/20... ', 'Training Step: 1311... ', 'Training loss: 1.4800... ', '6.1426 sec/batch')
('Epoch: 7/20... ', 'Training Step: 1312... ', 'Training loss: 1.5034... ', '6.1153 sec/batch')
('Epoch: 7/20... ', 'Training Step: 1313

('Epoch: 7/20... ', 'Training Step: 1389... ', 'Training loss: 1.4610... ', '6.0757 sec/batch')
('Epoch: 7/20... ', 'Training Step: 1390... ', 'Training loss: 1.5044... ', '6.1022 sec/batch')
('Epoch: 7/20... ', 'Training Step: 1391... ', 'Training loss: 1.4868... ', '6.0661 sec/batch')
('Epoch: 7/20... ', 'Training Step: 1392... ', 'Training loss: 1.5194... ', '6.1379 sec/batch')
('Epoch: 7/20... ', 'Training Step: 1393... ', 'Training loss: 1.4801... ', '6.0535 sec/batch')
('Epoch: 7/20... ', 'Training Step: 1394... ', 'Training loss: 1.4869... ', '6.1071 sec/batch')
('Epoch: 7/20... ', 'Training Step: 1395... ', 'Training loss: 1.4734... ', '6.0553 sec/batch')
('Epoch: 7/20... ', 'Training Step: 1396... ', 'Training loss: 1.4223... ', '6.2326 sec/batch')
('Epoch: 7/20... ', 'Training Step: 1397... ', 'Training loss: 1.4646... ', '5.9511 sec/batch')
('Epoch: 7/20... ', 'Training Step: 1398... ', 'Training loss: 1.4699... ', '5.9769 sec/batch')
('Epoch: 7/20... ', 'Training Step: 1399

('Epoch: 8/20... ', 'Training Step: 1475... ', 'Training loss: 1.4746... ', '6.1386 sec/batch')
('Epoch: 8/20... ', 'Training Step: 1476... ', 'Training loss: 1.4404... ', '6.2644 sec/batch')
('Epoch: 8/20... ', 'Training Step: 1477... ', 'Training loss: 1.4713... ', '5.8927 sec/batch')
('Epoch: 8/20... ', 'Training Step: 1478... ', 'Training loss: 1.4809... ', '6.1710 sec/batch')
('Epoch: 8/20... ', 'Training Step: 1479... ', 'Training loss: 1.5036... ', '6.0643 sec/batch')
('Epoch: 8/20... ', 'Training Step: 1480... ', 'Training loss: 1.4983... ', '6.1055 sec/batch')
('Epoch: 8/20... ', 'Training Step: 1481... ', 'Training loss: 1.4177... ', '6.2583 sec/batch')
('Epoch: 8/20... ', 'Training Step: 1482... ', 'Training loss: 1.4816... ', '6.1261 sec/batch')
('Epoch: 8/20... ', 'Training Step: 1483... ', 'Training loss: 1.4474... ', '6.0094 sec/batch')
('Epoch: 8/20... ', 'Training Step: 1484... ', 'Training loss: 1.4562... ', '5.8834 sec/batch')
('Epoch: 8/20... ', 'Training Step: 1485

('Epoch: 8/20... ', 'Training Step: 1561... ', 'Training loss: 1.4440... ', '6.1269 sec/batch')


KeyboardInterrupt: 

#### Saved checkpoints

Read up on saving and loading checkpoints here: https://www.tensorflow.org/programmers_guide/variables

In [18]:
tf.train.get_checkpoint_state('checkpoints')

model_checkpoint_path: "checkpoints/i1400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i1000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i1200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i1400_l512.ckpt"

## Sampling

Now that the network is trained, we'll can use it to generate new text. The idea is that we pass in a character, then the network will predict the next character. We can use the new one, to predict the next one. And we keep doing this to generate all new text. I also included some functionality to prime the network with some text by passing in a string and building up a state from that.

The network gives us predictions for each character. To reduce noise and make things a little less random, I'm going to only choose a new character from the top N most likely characters.



In [19]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [20]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    samples = [c for c in prime]
    model = CharRNN(len(vocab), lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

Here, pass in the path to a checkpoint and sample from the network.

In [21]:
tf.train.latest_checkpoint('checkpoints')

u'checkpoints/i1400_l512.ckpt'

In [22]:
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i1400_l512.ckpt
Farreds
the contented on the rather was the begterer, and the partic alowal of
the point of the mears, and was to the book, but this comprecelly seemed
to his what an instant in his hand, and she would be a sund
simplester.. And walked of the saying white was to that shoul he changed
that the moment, and he had anything, but he will to the sorn with his
weart that he there a men with the sould of this sair and say of them.

"You said I duding, he has maness in the more than he windes imporerely,
then they seemed to the something the care that say that he could not be
don't conevine, the privence of the talle talk to the counters
of a counting, which said at his mace.

"Yes, there the strunger, and him," the same, stranghing, he would have
and to see me."

"Well, it's nease, with a complete men," said She herestanion to hers his
confenced would be seemed to shave the some and as in the same that the
peesion, he could 

In [23]:
checkpoint = 'checkpoints/i200_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i200_l512.ckpt
Far ille olo the sher sor tit he tot her ood ase there sorthe wote ton sot han anthad teas han tand wothe sonld tise the ad ootil at an oot he ant ate hitt the son te se set ha she teral hhe whe to ote hor sinl her oot her sothes who wo hed sost oled ath the sed whe sat to satt ale hedend he wer ho tha lhe se on an he tat hes altit her sond ter horde non one thin olo to thin and timting of the tha led whe he ther ald te an all as ant on tout hher oon at ond toun hos, tound hand and, at inte whireg
te sorte the satisg as ate ans ot he sher onle wit atith he teres thind ald woth thisd she coreld
the her ose teel, alt ande ast oot heres, sod her an thes whad whas hosd and, tar and ward hard
nod ond thare he ade antese th te seresd sose an ond othe tes heed to tith ter sard totin te ther anl tite the and the sered andd hins setans toret ilter onse add hon te hor sosd he ad sote
s o her terer toth shesd onde hat allit al t

In [24]:
checkpoint = 'checkpoints/i600_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i600_l512.ckpt
Fard im,.

"And I was a laving to shime a disters firting of she
was thinc he his the porerted at have wate a custang and alligater. There
the breass with he come his tore the sers a way he had he
bratked her the fore simpent, whith
comering or with a stort himsed at the sames of shime offr it
shing, and so she
what he said," he had te seide to the sard the
conting on that things ate that a shis than was he that she was storded.

"I mante thay she how and hid not, shilly astered in once the
poons out that, thenes thas his and
thay to dore was in the rectast, and whe woned to to her she and theye sang to her ander the storning, and had be netrings as hass, and the pasting tine or the carte and, but
and seep hars that to the courdes, and ho cond to dound he hourd
sat into the sack he was this how her the
child.

"It a leve atened the sampen of the roms and wish that he doow an tho carlion, some thing, this
the chest and

In [25]:
checkpoint = 'checkpoints/i1200_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i1200_l512.ckpt
Farress
the mustrait of her at he were his say. The sent there was a parted a
dresters.

"You come a denge it won't comp to see that than some men to be sometely of the
moness at the planers. The plencal tarks of the some alrasing the complese
of the painter of the dand tom anowing of her hustered, a sund stancing the
servitions."

"Ah!" she he to some him. He went to him. "Yes," said Sergey Ivanovitch, "have hough
as this it wene suched it was that have no teer they talking to the sents
out. At a carrowader, that when she saw the collence work," she
said, said, but it say his face and he was and told her so mare to
this always with the poiding the clild children. She stought the stoot who was
drivile were and had say to be the suppress. The clees tanking and was a
the differtion, there would she husbent on the samital than this said they he was
began and holden was and shoulder seemed to say his bectul and tarking a